<a href="https://colab.research.google.com/github/supplient/bachelor_design/blob/equal_realize/Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Environment Setup

In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [2]:
!git clone https://github.com/supplient/bachelor_design.git
import os
os.chdir("bachelor_design")
!pwd

Cloning into 'bachelor_design'...
remote: Enumerating objects: 108, done.
remote: Counting objects: 100% (108/108), done.
remote: Compressing objects: 100% (70/70), done.
remote: Total 351 (delta 64), reused 71 (delta 38), pack-reused 243
Receiving objects: 100% (351/351), 5.99 MiB | 9.12 MiB/s, done.
Resolving deltas: 100% (195/195), done.
/content/bachelor_design


In [3]:
!git checkout equal_realize

Branch 'equal_realize' set up to track remote branch 'equal_realize' from 'origin'.
Switched to a new branch 'equal_realize'
Already up to date.


In [47]:
!git pull

remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 3 (delta 2), reused 3 (delta 2), pack-reused 0
Unpacking objects: 100% (3/3), done.
From https://github.com/supplient/bachelor_design
   9710746..519bd10  equal_realize -> origin/equal_realize
Updating 9710746..519bd10
Fast-forward
 epoch_checkpoint.py | 3 +++
 1 file changed, 3 insertions(+)


# Mount GDriver
Perpare Colab environment.

In [4]:
from driver_amount import addh

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive
[Colab] Using address head: /gdrive/My Drive


In [5]:
!pip install keras_bert

  Created wheel for keras-bert: filename=keras_bert-0.81.0-cp36-none-any.whl size=37913 sha256=7567f012ade7ad20014d32734ee6b952193364127e33ecbc8718181574cdb68e
  Stored in directory: /root/.cache/pip/wheels/bd/27/da/ffc2d573aa48b87440ec4f98bc7c992e3a2d899edb2d22ef9e
  Created wheel for keras-transformer: filename=keras_transformer-0.32.0-cp36-none-any.whl size=13266 sha256=236be3b492bee5aff95248e35853aa4882d973a7aa92953a54d673d16694576e
  Stored in directory: /root/.cache/pip/wheels/62/f0/ce/82fa5d024d5ef8e263f26a50dcee23820efe245680ce9c922a
  Created wheel for keras-pos-embd: filename=keras_pos_embd-0.11.0-cp36-none-any.whl size=7554 sha256=b7ad0387e4a4f81f271faf94a31365159232efe269cf90f7f30d135e8fcff3dd
  Stored in directory: /root/.cache/pip/wheels/5b/a1/a0/ce6b1d49ba1a9a76f592e70cf297b05c96bc9f418146761032
  Created wheel for keras-multi-head: filename=keras_multi_head-0.22.0-cp36-none-any.whl size=15371 sha256=0332526d89fc1129105cc3c756832d4e16f0e98593c51e5e5c79864999619d6e
  Stor

In [6]:
!pip install git+https://www.github.com/keras-team/keras-contrib.git

  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-okoqqfl9
  Running command git clone -q https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-okoqqfl9
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-cp36-none-any.whl size=101064 sha256=9d6fce0fed8f578608f6d43700e50290cd03784a0e57edc80e028c6e38362790
  Stored in directory: /tmp/pip-ephem-wheel-cache-k0iqdaaz/wheels/11/27/c8/4ed56de7b55f4f61244e2dc6ef3cdbaff2692527a2ce6502ba
Successfully built keras-contrib


In [0]:
import os
os.chdir("/usr/local/lib/python3.6/dist-packages/keras_contrib/layers/")
lines = []
with open("crf.py", "r") as fd:
  lines = fd.readlines()
lines[515] = "            mask2 = K.cast(K.concatenate([mask, K.cast(K.zeros_like(mask[:, :1]), mask.dtype)], axis=1),\n"
with open("crf.py", "w") as fd:
  fd.writelines(lines)

os.chdir("/content")

In [0]:
os.chdir("bachelor_design")

# Prepare Input

In [0]:
from cut_and_tag import cut_and_tag
import preprocess
import config

In [10]:
cut_seqs, char_seqs, tag_seqs = cut_and_tag(
    addh + config.DATA_PATH,
    addh + config.STOPWORDS_PATH
)
char_seqs, tag_seqs = preprocess.shuffle_twin(
    char_seqs,
    tag_seqs
)
token_id_seqs, segment_seqs, tag_id_seqs, tag_vocab = preprocess.preprocess(
    char_seqs, 
    tag_seqs,
    addh + config.BERT_VOCAB_PATH,
    SEQ_LEN=config.SEQ_LEN
    )

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.787 seconds.
Prefix dict has been built successfully.
Using TensorFlow backend.


In [0]:
# Split train and text
train_num = int(len(token_id_seqs) * 0.9)

test_token_id_seqs = token_id_seqs[train_num:]
test_segment_seqs = segment_seqs[train_num:]
test_tag_id_seqs = tag_id_seqs[train_num:]

train_token_id_seqs = token_id_seqs[0:train_num]
train_segment_seqs = segment_seqs[0:train_num]
train_tag_id_seqs = tag_id_seqs[0:train_num]

In [0]:
import json
with open(addh + config.TAG_VOCAB_PATH, "w") as fd:
    json.dump(tag_vocab, fd)

In [13]:
tag_vocab

{'': 0,
 'B-EI': 2,
 'B-EIF': 10,
 'B-EO': 4,
 'B-EQ': 8,
 'B-ILF': 6,
 'I-EI': 3,
 'I-EIF': 11,
 'I-EO': 5,
 'I-EQ': 9,
 'I-ILF': 7,
 'O': 1}

# Build Model
using BERT

In [0]:
import keras
import keras_bert 

In [15]:
bert_model = keras_bert.load_trained_model_from_checkpoint(
    # config_file
    addh + config.BERT_CONFIG_PATH, 
    # checkpoint file
    addh + config.BERT_CHECKPOINT_PATH, 
    # settings
    training=False, # Not train the whole model. Ignore NSP and MLM
    trainable=True

)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [0]:
from keras_contrib.layers import CRF

In [0]:
input_token = keras.layers.Input(shape=(config.SEQ_LEN,))
input_segment = keras.layers.Input(shape=(config.SEQ_LEN,))

bert_output = bert_model([input_token, input_segment])

crf_model = CRF(len(tag_vocab), sparse_target=True)

output = crf_model(bert_output)

model = keras.models.Model([input_token, input_segment], output)

In [18]:
model.compute_output_shape([(1, config.SEQ_LEN), (1, config.SEQ_LEN)])

(1, 512, 12)

In [0]:
bert_model.summary()

In [20]:
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 512)          0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 512)          0                                            
__________________________________________________________________________________________________
model_2 (Model)                 (None, 512, 768)     177262848   input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
crf_1 (CRF)                     (None, 512, 12)      9396        model_2[1][0]              

# Fine-tuning

In [21]:
model.compile(
    optimizer=keras.optimizers.Adam(lr=0.00004),
    loss=crf_model.loss_function,
    metrics=[crf_model.accuracy]
)

/usr/local/lib/python3.6/dist-packages/keras_contrib/layers/crf.py:346: UserWarning: CRF.loss_function is deprecated and it might be removed in the future. Please use losses.crf_loss instead.
  warnings.warn('CRF.loss_function is deprecated '
/usr/local/lib/python3.6/dist-packages/keras_contrib/layers/crf.py:353: UserWarning: CRF.accuracy is deprecated and it might be removed in the future. Please use metrics.crf_accuracy
  warnings.warn('CRF.accuracy is deprecated and it '


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [48]:
import importlib
importlib.reload(epoch_checkpoint)

<module 'epoch_checkpoint' from '/content/bachelor_design/epoch_checkpoint.py'>

In [0]:
import epoch_checkpoint
epoch_callback = epoch_checkpoint.EpochCheckpoint(
    addh + config.MODEL_PATH,
    addh + config.TRAIN_REC_PATH,
    1,
    [test_token_id_seqs, test_segment_seqs],
    test_tag_id_seqs,
    tag_vocab
    )

In [0]:
batch_size = 16
completed = False
import tensorflow as tf

In [0]:
while not completed:
  try:
    model.fit(
      x=[train_token_id_seqs, train_segment_seqs],
      y=[train_tag_id_seqs], 
      batch_size=batch_size, 
      epochs=30,
      verbose=1,
      validation_data=[[test_token_id_seqs, test_segment_seqs], [test_tag_id_seqs]],
      callbacks=[epoch_callback]
    )
    completed = True
  except tf.errors.ResourceExhaustedError as e:
    batch_size = int(batch_size/2)
    print("Batch Size too large, turn to " + str(batch_size))
    if batch_size < 1:
      raise e

Train on 7005 samples, validate on 779 samples
Epoch 1/30
7005/7005 [==============================] - 2438s 348ms/step - loss: 55.6910 - crf_viterbi_accuracy: 0.9030 - val_loss: 56.1400 - val_crf_viterbi_accuracy: 0.8825
Saving checkpint...
Predicing for matrics calculating...
779/779 [==============================] - 95s 122ms/step
Updating train record...
Epoch 2/30
7005/7005 [==============================] - 2442s 349ms/step - loss: 55.6308 - crf_viterbi_accuracy: 0.9212 - val_loss: 56.1451 - val_crf_viterbi_accuracy: 0.8757
Saving checkpint...
Predicing for matrics calculating...
779/779 [==============================] - 96s 124ms/step
Updating train record...
Epoch 3/30
7005/7005 [==============================] - 2455s 350ms/step - loss: 55.6038 - crf_viterbi_accuracy: 0.9295 - val_loss: 56.1804 - val_crf_viterbi_accuracy: 0.8793
Saving checkpint...
Predicing for matrics calculating...
779/779 [==============================] - 96s 124ms/step
Updating train record...
Epoch 4/

In [42]:
for key, value in epoch_callback.train_rec[1]["logs"].items():
  print(str(key) + ": " + str(type(key)) + ", " + str(type(value)))

val_loss: <class 'str'>, <class 'float'>
val_crf_viterbi_accuracy: <class 'str'>, <class 'float'>
loss: <class 'str'>, <class 'numpy.float64'>
crf_viterbi_accuracy: <class 'str'>, <class 'numpy.float32'>


In [0]:
model.save(addh + config.MODEL_PATH)